In [29]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import warnings
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [30]:
df = pd.read_excel('/kaggle/input/breast-cancer-wisconsin/cancer.xlsx')
data = df.rename(columns={'Column1': 'ID', 'Column2': 'Y', 'Column3': 'radius1', 'Column4': 'texture1', 'Column5': 'perimeter1', 
                     'Column6': 'area1', 'Column7': 'smoothness1', 'Column8': 'compactness1', 'Column9': 'concavity1', 'Column10': 'concave_points1',
                     'Column11': 'symmetry1', 'Column12': 'fractal_dimension1', 'Column13': 'radius2', 'Column14': 'texture2', 'Column15': 'perimeter2', 
                     'Column16': 'area2', 'Column17': 'smoothness2', 'Column18': 'compactness2', 'Column19': 'concavity2', 'Column20': 'concave_points2',
                     'Column21': 'symmetry2', 'Column22': 'fractal_dimension2', 'Column23': 'radius3', 'Column24': 'texture3', 'Column25': 'perimeter3', 
                     'Column26': 'area3', 'Column27':'smoothness3', 'Column28': 'compactness3', 'Column29': 'concavity3', 'Column30': 'concave_points3',
                     'Column31': 'symmetry3', 'Column32': 'fractal_dimension3'})
data['Y'] = data['Y'].replace({'B': 0, 'M': 1})
# features_to_normalize = ['var1', 'var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9', 'var0']
cols_to_normalize = data.columns[2:]

# Normalize the selected columns using Min-Max scaling
scaler = MinMaxScaler()
data[cols_to_normalize] = scaler.fit_transform(data[cols_to_normalize])
data

,ID,Y,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,842302,1,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,842517,1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,84300903,1,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,84348301,1,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,84358402,1,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,1,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,...,0.623266,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667
565,926682,1,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,...,0.560655,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315
566,926954,1,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,...,0.393099,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909
567,927241,1,0.644564,0.663510,0.665538,0.475716,0.588336,0.790197,0.823336,0.755467,...,0.633582,0.730277,0.668310,0.402035,0.619626,0.815758,0.749760,0.910653,0.497142,0.452315


## Generate knockoff variables 

In [31]:
import numpy as np
from scipy.sparse.linalg import eigsh

## some useful functions
def get_arccos(X):
    # X is a 2-d array
    
    n, p = X.shape
    cos_a = np.zeros([n, n, n])
    
    for r in range(n):
        
        xr = X[r]
        X_r = X - xr
        cross = np.dot(X_r, X_r.T)
        row_norm = np.sqrt(np.sum(X_r**2, axis = 1))
        outer_norm = np.outer(row_norm, row_norm)
        
        zero_idx = (outer_norm == 0.)
        outer_norm[zero_idx] = 1.
        cos_a_kl = cross / outer_norm
        cos_a_kl[zero_idx] = 0.

        cos_a[:,:,r] = cos_a_kl
        
    cos_a[cos_a > 1] = 1.
    cos_a[cos_a < -1] = -1.
    a = np.arccos(cos_a)

    a_bar_12 = np.mean(a, axis = 0, keepdims = True)
    a_bar_02 = np.mean(a, axis = 1, keepdims = True)
    a_bar_2  = np.mean(a, axis = (0,1), keepdims = True)
    A = a - a_bar_12 - a_bar_02 + a_bar_2
        
    return a, A

def get_arccos_1d(X):
    # X is a 1-d array
    
    X = np.squeeze(X)
    Y = X[:,None] - X
    Z = Y.T[:,:,None]*Y.T[:,None]
    n = len(X)
    
    a = np.zeros([n, n, n])
    a[Z == 0.] = np.pi/2.
    a[Z < 0.] = np.pi
    
    a = np.transpose(a, (1,2,0))
    
    #a = Z[Z>0.]*0. + Z[Z==0.]*np.pi/2. + Z[Z<0.]*np.pi

    a_bar_12 = np.mean(a, axis = 0, keepdims = True)
    a_bar_02 = np.mean(a, axis = 1, keepdims = True)
    a_bar_2  = np.mean(a, axis = (0,1), keepdims = True)
    A = a - a_bar_12 - a_bar_02 + a_bar_2
    
    return a, A

def orthonormalize(X):
    # X is a 2-d array
    # output: Gram-Schmidt orthogonalization of X
    
    n, p = X.shape
    Y = np.zeros([n,p])
    Y[:,0] = X[:,0]/np.sqrt(np.sum(X[:,0]**2))
    
    for j in range(1,p):
        
        Yj = Y[:,range(j)]
        xj = X[:,j]
        w = np.dot(xj, Yj)
        xj_p = np.sum(w*Yj, axis = 1)
        yj = xj - xj_p
        yj = yj/np.sqrt(np.sum(yj**2))
        
        Y[:,j] = yj
        
    return Y

# Main functions
def projection_corr(X, Y):
    # X, Y are 2-d array
    
    nx, p = X.shape
    ny, q = Y.shape
    
    if nx == ny:
        n = nx
    else:
        raise ValueError("sample sizes do not match.")
        
    a_x, A_x = get_arccos(X)
    a_y, A_y = get_arccos(Y)
    
    S_xy = np.sum(A_x * A_y) / (n**3)
    S_xx = np.sum(A_x**2) / (n**3)
    S_yy = np.sum(A_y**2) / (n**3)
    
    if S_xx * S_yy == 0.:
        corr = 0.
    else:
        corr = np.sqrt( S_xy / np.sqrt(S_xx * S_yy) )
    
    return corr

def projection_corr_1d(X, Y):
    
    nx, p = X.shape
    ny, q = Y.shape
    
    if nx == ny:
        n = nx
    else:
        raise ValueError("sample sizes do not match.")
        
    a_x, A_x = get_arccos_1d(X)
    a_y, A_y = get_arccos_1d(Y)
    
    S_xy = np.sum(A_x * A_y) / (n**3)
    S_xx = np.sum(A_x**2) / (n**3)
    S_yy = np.sum(A_y**2) / (n**3)
    
    if S_xx * S_yy == 0.:
        corr = 0.
    else:
        corr = np.sqrt( S_xy / np.sqrt(S_xx * S_yy) )
    
    return corr

def projection_corr_1dy(X, Y):
    
    nx, p = X.shape
    ny, q = Y.shape
    
    if nx == ny:
        n = nx
    else:
        raise ValueError("sample sizes do not match.")
        
    a_x, A_x = get_arccos(X)
    a_y, A_y = get_arccos_1d(Y)
    
    S_xy = np.sum(A_x * A_y) / (n**3)
    S_xx = np.sum(A_x**2) / (n**3)
    S_yy = np.sum(A_y**2) / (n**3)
    
    if S_xx * S_yy == 0.:
        corr = 0.
    else:
        corr = np.sqrt( S_xy / np.sqrt(S_xx * S_yy) )
    
    return corr

def get_equi_features(X):
    # X is 2-d array
    
    n, p = X.shape
    scale = np.sqrt(np.sum(X**2, axis=0))
    Xstd = X / scale
    sigma = np.dot(Xstd.T, Xstd)
    sigma_inv = np.linalg.inv(sigma)
    lambd_min = eigsh(sigma, k=1, which='SA')[0].squeeze()
    sj = np.min([1., 2.*lambd_min])
    sj = sj - 0.00001
    
    mat_s = np.diag([sj]*p)
    A = 2*mat_s - sj*sj*sigma_inv
    C = np.linalg.cholesky(A).T
    
    Xn = np.random.randn(n, p)
    XX = np.hstack([Xstd, Xn])
    XXo = orthonormalize(XX)
    U = XXo[:,range(p,2*p)]
    
    Xnew = np.dot(Xstd,  np.eye(p) - sigma_inv*sj) + np.dot(U,C)
    return Xnew

In [32]:
X = data.iloc[:,2:]
Y = data.iloc[:,1]
X = (X - X.mean()) / X.std()
X_knockoff = get_equi_features(X)

In [33]:
X_knockoff_df = pd.DataFrame(X_knockoff)
column_names = ['radius1', 'texture1', 'perimeter1', 
                'area1', 'smoothness1', 'compactness1', 'concavity1', 'concave_points1',
                'symmetry1', 'fractal_dimension1', 'radius2', 'texture2', 'perimeter2', 
                'area2', 'smoothness2', 'compactness2', 'concavity2', 'concave_points2',
                'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'perimeter3', 
                'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3',
                'symmetry3', 'fractal_dimension3']
X_knockoff_df.columns = column_names
feature = pd.concat([X,X_knockoff_df],axis = 1)
dataset1 =  pd.concat([feature,data['Y']],axis = 1)
dataset1
# dataset1.to_csv('dataset1_for_DeepPINK.csv', index=False)

,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,fractal_dimension1,...,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3,Y
0,1.096100,-2.071512,1.268817,0.983510,1.567087,3.280628,2.650542,2.530249,2.215566,2.253764,...,-0.056998,0.095378,0.083816,0.055787,0.108451,0.088815,0.096621,0.114731,0.080628,1
1,1.828212,-0.353322,1.684473,1.907030,-0.826235,-0.486643,-0.023825,0.547662,0.001391,-0.867889,...,-0.014257,0.064647,0.077837,-0.014914,-0.017679,-0.006220,0.043501,-0.010305,0.011823,1
2,1.578499,0.455786,1.565126,1.557513,0.941382,1.052000,1.362280,2.035440,0.938859,-0.397658,...,0.000420,0.057754,0.060255,0.022150,0.044780,0.034473,0.081035,0.048529,0.007839,1
3,-0.768233,0.253509,-0.592166,-0.763792,3.280667,3.399917,1.914213,1.450431,2.864862,4.906602,...,0.005121,-0.011096,-0.022331,0.141655,0.164464,0.084088,0.092006,0.252517,0.207118,1
4,1.748758,-1.150804,1.775011,1.824624,0.280125,0.538866,1.369806,1.427237,-0.009552,-0.561956,...,-0.063168,0.056116,0.050540,0.007508,-0.013955,0.025858,0.031509,-0.035818,-0.016585,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,2.109139,0.720838,2.058974,2.341795,1.040926,0.218868,1.945573,2.318924,-0.312314,-0.930209,...,0.004531,0.071203,0.084794,0.016522,-0.011060,0.027293,0.067770,-0.056440,-0.029971,1
565,1.703356,2.083301,1.614511,1.722326,0.102368,-0.017817,0.692434,1.262558,-0.217473,-1.057681,...,0.085282,0.059169,0.062881,-0.027549,-0.014854,0.009683,0.031218,-0.021199,-0.042582,1
566,0.701667,2.043775,0.672084,0.577445,-0.839745,-0.038646,0.046547,0.105684,-0.808406,-0.894800,...,0.058306,0.024187,0.019538,-0.033934,0.012982,0.016183,0.017781,-0.046413,-0.012294,1
567,1.836725,2.334403,1.980781,1.733693,1.524426,3.269267,3.294046,2.656528,2.135315,1.042778,...,0.095843,0.097124,0.070226,0.060661,0.164273,0.135185,0.096564,0.080864,0.092753,1


## Create network

### OL

In [34]:
lambda_array = np.zeros((60, 10))
lambda_array2 = np.zeros((10, 1))
input_dim = 60

# Use tqdm for a one-line progress bar
for i in tqdm(np.arange(0, 1, 0.01)):
    model = keras.Sequential()
    model.add(layers.Dense(10, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(i)))
    #model.add(layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(i)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(feature, Y, epochs=50, batch_size=32, verbose=0)

    first_layer = model.layers[0]
    weights, biases = first_layer.get_weights()
    
    for j in range(min(len(weights), len(lambda_array))):
        for n in range(min(len(weights[j]), len(lambda_array[j]))):
            if abs(weights[j][n]) < 5e-4 and lambda_array[j][n] == 0:
                lambda_array[j][n] = i
    
    # Calculate the Zi in second layer
    second_layer = model.layers[1]
    weights, biases = second_layer.get_weights()
    #print(f"Layer: {layer.name}, Weights: {weights}")
#     print(weights)
#     for j in range(len(weights)):
#         for n in range(len(weights[j])):
#             if abs(weights[j][n]) < 5e-4 and lambda_array2[j][n] == 0 :
# #                 print(j,n)
#                 lambda_array2[j][n] = i
print(lambda_array)

100%|██████████| 100/100 [04:28<00:00,  2.69s/it]

[[0.26 0.12 0.06 0.19 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.26 0.11 0.06 0.17 0.21 0.08 0.07 0.06 0.23 0.14]
 [0.26 0.11 0.06 0.17 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.26 0.12 0.06 0.17 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.26 0.11 0.06 0.17 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.26 0.04 0.06 0.1  0.12 0.03 0.1  0.06 0.14 0.1 ]
 [0.26 0.11 0.06 0.26 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.26 0.11 0.06 0.19 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.26 0.11 0.06 0.13 0.21 0.08 0.1  0.06 0.14 0.14]
 [0.26 0.11 0.06 0.19 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.26 0.11 0.06 0.17 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.1  0.11 0.06 0.08 0.16 0.08 0.07 0.06 0.04 0.08]
 [0.26 0.11 0.06 0.17 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.26 0.11 0.06 0.17 0.21 0.08 0.1  0.06 0.23 0.14]
 [0.15 0.11 0.06 0.13 0.15 0.08 0.07 0.06 0.08 0.14]
 [0.26 0.11 0.06 0.13 0.21 0.08 0.07 0.06 0.23 0.14]
 [0.15 0.03 0.06 0.13 0.15 0.08 0.08 0.04 0.08 0.06]
 [0.26 0.11 0.06 0.17 0.21 0.08 0.07 0.06 0.14 0.14]
 [0.26 0.11 0.06 0.17 0.21 0.08 0.07 0.06 0.14

In [35]:
non_zero_lambda_array = lambda_array[np.nonzero(lambda_array)]
min_non_zero_lambda = np.min(np.abs(non_zero_lambda_array))
max_lambda = np.max(abs(lambda_array))
#for i in lambda_array:
    #print(i)
print(f"min_lambda (except 0): {min_non_zero_lambda}")
print(f"max_lambda: {max_lambda}")
count_min_non_zero_lambda = np.count_nonzero(np.abs(lambda_array) == 0)
print(f"number of 0: {count_min_non_zero_lambda}")

min_lambda (except 0): 0.01
max_lambda: 0.28
number of 0: 0


In [36]:
lambda_array2

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [42]:
def compute_W1(lambda_array):
    lambda_array = np.where(lambda_array == 0, 2, lambda_array)
    row_sums1 = np.sum(lambda_array, axis=1)
    W1 = []
    for i in range(0,int(0.5*len(row_sums1)),1):
        if row_sums1[i]>=row_sums1[i+30]:
            W1.append(row_sums1[i])
        else:
            W1.append(-row_sums1[i+30])
    return W1,row_sums1

def select_variables1(W,target_q):
    T = []
    for i in W1:
        count1 = 0
        count2 = 0
        t = abs(i)
        for j in W1:
            if j < -t:
                count1 = count1+1
            if j >= t:
                count2 = count2 +1
        q = (count1)/(max(count2,1))
    #     print(count1,count2,q)
        if q <= target_q:
            T.append(t)
    threshold = min(T)
    selected_vars = [i+1 for i in range(len(W)) if W[i] >= threshold]
    return selected_vars, threshold

# Sample usage:
W1, row_sums1 = compute_W1(lambda_array)
selected_vars1, threshold_value1 = select_variables1(W1,1)


In [41]:
def compute_W2(model, lambda_array):
    # Get weights of the layers
    weight_list = [layer.get_weights()[0] for layer in model.layers if 'dense' in layer.name]
    
    # Compute the dot product across all layers
    w_m = weight_list[0]
#     print(w_m)
    for weight in weight_list[1:]:
        w_m = np.dot(w_m, weight)
#         print(w_m)
    # Adjust lambda_array values
    lambda_array_adj = np.where(lambda_array == 0, 2, lambda_array)
    row_sums = np.sum(lambda_array_adj, axis=1)
    #print(len(w_m))
    Z = [row_sums[i]*w_m[i] for i in range(len(w_m))]
    W = [Z[i]**2/10000 - Z[i+30]**2/10000 for i in range(int(len(Z)/2))]
#     print(Z[i]**2)
#     print(row_sums)
    return W,Z

def select_variables2(W,target_q):
    T = []
    for i in W:
        count1 = len([j for j in W if j < -abs(i)])
        count2 = len([j for j in W if j >= abs(i)])
        q = count1 / max(count2, 1)
        if q < target_q:
            T.append(abs(i))
    
    threshold = min(T)
    selected_vars = [i+1 for i in range(len(W)) if W[i] >= threshold]
    return selected_vars, threshold

# Sample usage:

model_2 = keras.Sequential()
model_2.add(layers.Dense(10, activation='relu'))
model_2.add(layers.Dense(1, activation='sigmoid'))
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_2.fit(feature, Y, epochs=50, batch_size=32, verbose=0)
W2,Z = compute_W2(model_2, lambda_array)
selected_vars2, threshold_value2 = select_variables2(W2,0.01)
selected_vars2, threshold_value2

([7, 13, 16, 20, 21, 22, 24, 25, 28, 29], array([0.00012937], dtype=float32))